In [1]:
pip install datasets

  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
from datasets import load_dataset

In [15]:
cbt_dataset = load_dataset('cbt','CN')

Generating train split:   0%|          | 0/120769 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset cbt downloaded and prepared to C:/Users/34941/.cache/huggingface/datasets/cbt/CN/1.1.0/dc4451a8a4b50cebb78fdb19fa9f964b27fcdcef915467b8b7055a3a8d8cef7b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
train_set = cbt_dataset['train']
val_set = cbt_dataset['validation']
test_set = cbt_dataset['test']

In [19]:
train_set[0]

{'sentences': ['With almost everything else to make them happy , they wanted one thing : they had no children .',
  'This vexed the king even more than the queen , who was very clever and learned , and who had hated dolls when she was a child .',
  'However , she , too in spite of all the books she read and all the pictures she painted , would have been glad enough to be the mother of a little prince .',
  'The king was anxious to consult the fairies , but the queen would not hear of such a thing .',
  'She did not believe in fairies : she said that they had never existed ; and that she maintained , though The History of the Royal Family was full of chapters about nothing else .',
  'Well , at long and at last they had a little boy , who was generally regarded as the finest baby that had ever been seen .',
  'Even her majesty herself remarked that , though she could never believe all the courtiers told her , yet he certainly was a fine child -- a very fine child .',
  'Now , the time d